# Confusion Matrix Generator for Syllabification and Tone Engines Evaluation
This notebook processes two zipped datasets containing validated syllabification and tone labelling wordlists across multiple languages. It generates confusion matrix tables representing system versus expert agreement.

In [2]:
# Upload zipped syllables and tones output files for evaluation
from google.colab import files
uploaded = files.upload()

Saving tone_labelling_output_result.zip to tone_labelling_output_result.zip


In [3]:
#  Import useful Python libraries
import zipfile
from pathlib import Path
import pandas as pd
import os

In [4]:
# Define paths to your ZIP files
syllable_zip = 'syllabified_output_result.zip'
tone_zip = 'tone_labelling_output_result.zip'
syllable_extract_path = 'syllable_data'
tone_extract_path = 'tone_data'

# Extract syllable files
with zipfile.ZipFile(syllable_zip, 'r') as zip_ref:
    zip_ref.extractall(syllable_extract_path)

# Extract tone files
with zipfile.ZipFile(tone_zip, 'r') as zip_ref:
    zip_ref.extractall(tone_extract_path)


In [5]:
# Syllabification Confusion Matrix
syllable_files = list(Path(syllable_extract_path).rglob('*.xlsx'))
syllable_dfs = []

def classify_syllable(row):
    human_val = row.get('Human-in-the-loop Validation') or row.get('Human-in-the-loop Vaidation')
    if not isinstance(human_val, str): return 'Other'
    if human_val.strip() == 'Validated': return 'TN'
    elif 'Unsupported syllable structure' in human_val or 'Unspecified syllable structure' in human_val: return 'FN'
    elif 'correctly syllabified' in human_val and 'Character' in human_val: return 'FP'
    elif 'Character' in human_val: return 'TP'
    return 'Other'

for file in syllable_files:
    df = pd.read_excel(file)
    if 'Human-in-the-loop Vaidation' in df.columns:
        df.rename(columns={'Human-in-the-loop Vaidation': 'Human-in-the-loop Validation'}, inplace=True)
    df['Language'] = file.stem
    df['Classification'] = df.apply(classify_syllable, axis=1)
    syllable_dfs.append(df)

syllable_combined = pd.concat(syllable_dfs, ignore_index=True)
syllable_conf_matrix = syllable_combined.groupby('Language')['Classification'].value_counts().unstack(fill_value=0).reset_index()
for col in ['TP', 'FP', 'FN', 'TN']: syllable_conf_matrix[col] = syllable_conf_matrix.get(col, 0)
syllable_conf_matrix['Total'] = syllable_conf_matrix[['TP', 'FP', 'FN', 'TN']].sum(axis=1)
syllable_conf_matrix = syllable_conf_matrix[['Language', 'TP', 'FP', 'FN', 'TN', 'Total']]
syllable_conf_matrix.to_excel('HM_Syllables_Confusion_Matrix.xlsx', index=False)
syllable_conf_matrix.head()

Classification,Language,TP,FP,FN,TN,Total
0,Anaan,0,0,0,106,106
1,Edo,7,0,21,78,106
2,Efik,8,16,8,76,108
3,Ejagham,3,0,40,65,108
4,Ekid,0,2,0,105,107


In [6]:
# Tone Evaluation Confusion Matrix
tone_files = list(Path(tone_extract_path).rglob('*.xlsx'))
tone_dfs = []

def classify_tone(row):
    remarks = row['Remarks']
    verification = row['Tone Verification']
    if remarks == 'tone labels properly marked' and verification == 'Tone label(s) recognized': return 'TN'
    elif 'less tone labels' in remarks and 'not recognized' in verification: return 'TP'
    elif 'tone labels properly marked' in remarks and 'not recognized' in verification: return 'FP'
    elif 'more tone labels' in remarks and verification == 'Tone label(s) recognized': return 'FN'
    return 'Other'

for file in tone_files:
    df = pd.read_excel(file)
    df['Language'] = file.stem
    df['Classification'] = df.apply(classify_tone, axis=1)
    tone_dfs.append(df)

tone_combined = pd.concat(tone_dfs, ignore_index=True)
tone_conf_matrix = tone_combined.groupby('Language')['Classification'].value_counts().unstack(fill_value=0).reset_index()
for col in ['TP', 'FP', 'FN', 'TN']: tone_conf_matrix[col] = tone_conf_matrix.get(col, 0)
tone_conf_matrix['Total'] = tone_conf_matrix[['TP', 'FP', 'FN', 'TN']].sum(axis=1)
tone_conf_matrix = tone_conf_matrix[['Language', 'TP', 'FP', 'FN', 'TN', 'Total']]
tone_conf_matrix.to_excel('Tones_Confusion_Matrix.xlsx', index=False)
tone_conf_matrix.head()

Classification,Language,TP,FP,FN,TN,Total
0,Anaan,0,0,3,86,89
1,Edo,0,0,2,104,106
2,Efiik,0,0,19,85,104
3,Ejagham,0,1,2,97,100
4,Ekid,0,0,11,91,102
